## CS536: Perceptrons
#### Done by - Vedant Choudhary, vc389
In the usual way, we need data that we can fit and analyze using perceptrons. Consider generating data points (X, Y) in the following way:
- For $i = 1,....,k-1$, let $X_i ~ N(0, 1)$ (i.e. each $X_i$ is an i.i.d. standard normal)
- For $i = k$, generate $X_k$ in the following way: let $D ~ Exp(1)$, and for a parameter $\epsilon > 0$ take

$X_k = (\epsilon + D)$ with probability 1/2

$X_k = -(\epsilon + D)$ with probability 1/2

The effect of this is that while $X_1,...X_{k-1}$ are i.i.d. standard normals, $X_k$ is distributed randomly with some gap (of size $2\epsilon$ around $X_k = 0$. We can then classify each point according to the following:

$Y = 1$ with probability 1/2

$Y = -1$ with probability 1/2

We see that the class of each data point is determined entirely by the value of the $X_k$ feature

#### 1. Show that there is a perceptron that correctly classifies this data. Is this perceptron unique? What is the ‘best’ perceptron for this data set, theoretically?

In [2]:
# Importing required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pprint
from tqdm import tqdm

%matplotlib inline

In [3]:
# Creating X (feature) vectors for the data
def create_data(k, m, D, epsilon):
    X_k_minus_1 = np.random.normal(0, 1, (m,k-1))
    X_k = []
    for i in range(m):
        temp = np.random.choice(2, 1, p=[0.5,0.5])
#         print(temp)
        if temp == 1:
            X_k.append(epsilon + D)
        else:
            X_k.append(-(epsilon + D))
    X_k = np.asarray(X_k).reshape((1,m))
#     print(X_k_minus_1)
#     print(X_k)
    return np.concatenate((X_k_minus_1, X_k.T), axis=1)

# Creating target column for the data
def create_y(X, m):
    y = []
    for i in range(m):
        if X[i][-1] > 0:
            y.append(1)
        else:
            y.append(-1)
    return y

# Combining all the sub data points into a dataframe
def create_dataset(k, m, epsilon, D):
    X = np.asarray(create_data(k, m, epsilon, D))
    y = np.asarray(create_y(X, m)).reshape((m,1))

    # Training data is an appended version of X and y arrays
    data = pd.DataFrame(np.append(X, y, axis=1), columns=["X" + str(i) for i in range(1,k+1)]+['Y'])
    return data

In [4]:
# Global Variables - k = 20, m = 100, epsilon = 1
k, m, epsilon = 20, 100, 1
D = float(np.random.exponential(1, 1))

train_data = create_dataset(k, m, epsilon, D)
train_data.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X12,X13,X14,X15,X16,X17,X18,X19,X20,Y
0,-0.052815,1.041992,-0.317840,0.461445,-1.219667,1.736444,0.074190,0.521638,-0.096941,0.018287,...,-0.929177,-0.244840,-1.448114,-1.042703,1.376720,0.850087,-0.245150,0.356574,-4.080852,-1.0
1,0.506920,0.383559,-1.389646,-0.545733,-0.189458,0.867507,0.378216,-0.898799,0.644921,0.929950,...,0.044940,0.388951,0.300723,0.922243,-1.963112,0.987887,0.339391,-1.914598,4.080852,1.0
2,-1.231154,1.529727,-0.058887,-1.401355,-1.128395,-1.071578,-1.037940,-0.101086,-0.632556,1.178592,...,-1.061076,-1.232727,1.315680,0.749097,-0.641943,-0.099577,0.955176,-1.204766,4.080852,1.0
3,-0.924930,0.245410,0.743430,-0.766712,-0.442100,-0.738759,-0.880178,1.090554,0.495343,-0.161814,...,1.370903,-0.796894,0.794242,-0.729538,-1.877562,1.651299,-0.115813,1.759965,-4.080852,-1.0
4,0.238222,0.058716,-0.602490,-0.659524,0.759971,1.450825,-0.602260,-0.263689,1.305453,-0.855617,...,-0.150832,2.051842,1.531062,0.705523,1.872988,1.488366,-1.377275,-0.577903,-4.080852,-1.0


In [24]:
# Class for Perceptron
class Perceptron():
    def __init__(self):
        pass
    
    def sign_function(self, data_vec):
        return np.array([1 if val >= 1 else -1 for val in data_vec])[:, np.newaxis]
    
    def pla(self, data):
        X = np.asarray(data.iloc[:,:-1])
        y = np.asarray(data.iloc[:,-1:])
        num_samples, num_features = X.shape
#         Initialize weight and bias parameters
        self.w = np.zeros(shape=(num_features, 1))
        self.bias = 0
#         Add check condition if y != f_x ever
        while True:
#             Calculate the mapping function f(x)
            f_x = self.sign_function(np.dot(X, self.w) + self.bias)
#             Compute weights if f_x != y
            for i in range(num_samples):
                if f_x[i] != y[i]:
                    self.w += np.dot(X[i].T.reshape((num_features, 1)), y[i].reshape((1,1)))
                    self.bias += y[i]
                
            if np.array_equal(y, f_x):
                break
                    
        return self.w, self.bias
    
    '''
    Predicts the target value based on a data vector
    Input - a single row of dataset or a single X vector, decision tree
    Return - predicted value
    '''
    def predict(self, instance_data):
        prediction = self.sign_function(np.dot(self.w.T, np.asarray(instance_data).reshape((20,1))) + self.bias)
        return prediction   
    
    '''
    Predicts the target value and then calculates error based on the predictions
    Input - dataset, decision tree built
    Return - error
    '''
    def fit(self, data):
        error = 0
        for i in range(len(data)):
            prediction = self.predict(data.iloc[i][:-1])
            if prediction != data.iloc[i][-1]:
                error += 1
        return error/len(data) 
        

In [25]:
perceptron = Perceptron()
final_w, final_b = perceptron.pla(train_data)
print(final_w)
print(final_b)

[[-10.25373354]
 [ -3.45473026]
 [  4.14668222]
 [-10.81635368]
 [ -5.58331826]
 [  0.79608837]
 [ -5.93601992]
 [ -1.63990473]
 [  3.63995818]
 [ -9.70885884]
 [  4.72328779]
 [ 13.90303297]
 [  7.08709852]
 [ -4.69182138]
 [ -3.33967112]
 [ -8.14006658]
 [  9.43349819]
 [ 11.43870612]
 [ -3.53964662]
 [179.55747274]]
[44.]


In [27]:
error = perceptron.fit(train_data)
error

0.0

array([[-10.25373354],
       [ -3.45473026],
       [  4.14668222],
       [-10.81635368],
       [ -5.58331826],
       [  0.79608837],
       [ -5.93601992],
       [ -1.63990473],
       [  3.63995818],
       [ -9.70885884],
       [  4.72328779],
       [ 13.90303297],
       [  7.08709852],
       [ -4.69182138],
       [ -3.33967112],
       [ -8.14006658],
       [  9.43349819],
       [ 11.43870612],
       [ -3.53964662],
       [179.55747274]])